In [22]:
import praw
import pandas as pd
import numpy as np

import unicodedata
import string
import re
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import  SnowballStemmer
from nltk.stem import WordNetLemmatizer
from matplotlib import pyplot as plt

In [4]:
reddit = praw.Reddit(client_id='5B5OQI5BAIUa1w', client_secret='5eShI_61NRHwaw1lKNzIeMUG_lwQ0Q', user_agent='Reddit WebScrapping')

In [5]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit('MachineLearning').hot(limit=10)
for post in hot_posts:
    print(post.title)

[D] Simple Questions Thread November 22, 2020
[D] Machine Learning - WAYR (What Are You Reading) - Week 100
[D] Senior researcher here; introducing deep learning to a field which doesn't have it. Suggestions on how to deal with bad reviews
[R] AlphaFold 2
[D] AAAI 2021 Paper Acceptance Result
[P] 3 Ways to Classify Drum Sounds
[P] PolyFuzz - Fuzzy string matching, grouping, and evaluation in one
[D] Chess reinforcement learning
[D] Looking for folks who want to help each other with advanced ML mock interviews
[R] If AI papers were pharmaceuticals... A parody commercial for Facebook's NeurIPS 2020 paper


In [6]:
hot_posts = reddit.subreddit('all').hot(limit=10)
for post in hot_posts:
    print(post.title)

A sea turtle’s sex is determined by the temperature of the sand it’s born in; rising global temperatures mean that female green sea turtles now outnumber males 116 to 1.
What's yours?
throat goat
Road rage about to be like a COD lobby
Congress is briefly reconvening, under pressure from Trump and Biden to pass a COVID-19 stimulus bill after months of gridlock
what? i am just here licking my paws. i don't know who did it
Gamer moms for the win!
Hello world, 15 minutes old
Should we listen to Donald "Rake the Forests" Trump or people with actual science degrees?
TIL James Earl Jones decided to go uncredited for his role of voicing Darth Vader because he considered David Prowse’s performance inside the Vader costume to be the more defining of the two performances.


In [7]:
type(hot_posts)

praw.models.listing.generator.ListingGenerator

In [17]:
posts = []
all_hot = reddit.subreddit('all')
for post in all_hot.hot(limit=3000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [18]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,I couldn't be more proud of her. After a lot o...,18772,k411lv,gaming,https://i.redd.it/mx5qbdll1f261.jpg,489,,1.606788e+09
1,FCC Chairman Ajit Pai will step down on Januar...,81513,k3yin8,news,https://www.cnbc.com/2020/11/30/fcc-chairman-a...,3941,,1.606781e+09
2,LPT: No matter how much your workplace pushes ...,40132,k3ywxq,LifeProTips,https://www.reddit.com/r/LifeProTips/comments/...,1798,"Unfortunately, I learned this the hard way. I ...",1.606782e+09
3,Mark Kelly to be sworn in as senator on Wednesday,27587,k3xnka,politics,https://thehill.com/homenews/senate/527946-mar...,721,,1.606778e+09
4,"Today I turn 50, but here I am at 18- summer o...",29983,k3xrwi,OldSchoolCool,https://i.redd.it/w92l5gtdae261.jpg,822,,1.606779e+09
...,...,...,...,...,...,...,...,...
2995,Oops 😬 [F],215,k3y5o8,RealPublicNudity,https://i.redd.it/hhlt4hfsde261.jpg,0,,1.606780e+09
2996,no meme just an entire spongebob episode,215,k3yba1,NoRules,https://v.redd.it/r5jodes2fe261,33,,1.606780e+09
2997,Suprised this hasn't been used for any memes,2837,k3pmfg,reddeadredemption2,https://i.redd.it/g518s1kicb261.png,40,,1.606743e+09
2998,Palantir Adds $17 Billion in Value in Best Wee...,242,k3xe90,StockMarket,https://www.reddit.com/r/StockMarket/comments/...,50,Palantir Technologies Inc. posted its best we...,1.606777e+09


In [24]:
docs = posts['title'].tolist()

In [26]:
for i,doc in enumerate(docs):
    docs[i] = unicodedata.normalize('NFKD', doc).encode('ASCII', 'ignore').decode('utf8')

In [27]:
docs

["I couldn't be more proud of her. After a lot of blood sweat and tears...but mostly FUN, my mother finished her first Dark Souls game.",
 'FCC Chairman Ajit Pai will step down on January 20',
 'LPT: No matter how much your workplace pushes "team building" and "family culture" - remember, they\'re not your friends and it\'s still a workplace.',
 'Mark Kelly to be sworn in as senator on Wednesday',
 'Today I turn 50, but here I am at 18- summer of 1989',
 'The kids have been asking, dad is delivering. Christmas is going to be great...',
 'When you have 100% control over bike and no obstacle can stop you.',
 "What's yours?",
 'Hes solved the case, we can all move on now...',
 'TIl that the real life Top Gun flight school became so sick and tired of the constant movie references that they imposed a $5 fine to any staff member that quotes the movie',
 '[OC] USA COVID cases syringe',
 'They grow up so fast ',
 'Madlad....',
 'Hope he enjoys his bragging rights!',
 'Those are some good looki

In [28]:
for i,doc in enumerate(docs):
    docs[i] = doc.lower()  

In [29]:
docs

["i couldn't be more proud of her. after a lot of blood sweat and tears...but mostly fun, my mother finished her first dark souls game.",
 'fcc chairman ajit pai will step down on january 20',
 'lpt: no matter how much your workplace pushes "team building" and "family culture" - remember, they\'re not your friends and it\'s still a workplace.',
 'mark kelly to be sworn in as senator on wednesday',
 'today i turn 50, but here i am at 18- summer of 1989',
 'the kids have been asking, dad is delivering. christmas is going to be great...',
 'when you have 100% control over bike and no obstacle can stop you.',
 "what's yours?",
 'hes solved the case, we can all move on now...',
 'til that the real life top gun flight school became so sick and tired of the constant movie references that they imposed a $5 fine to any staff member that quotes the movie',
 '[oc] usa covid cases syringe',
 'they grow up so fast ',
 'madlad....',
 'hope he enjoys his bragging rights!',
 'those are some good looki

In [30]:
for i,doc in enumerate(docs):
    docs[i] = word_tokenize(doc) # method in nltk

In [31]:
docs

[['i',
  'could',
  "n't",
  'be',
  'more',
  'proud',
  'of',
  'her',
  '.',
  'after',
  'a',
  'lot',
  'of',
  'blood',
  'sweat',
  'and',
  'tears',
  '...',
  'but',
  'mostly',
  'fun',
  ',',
  'my',
  'mother',
  'finished',
  'her',
  'first',
  'dark',
  'souls',
  'game',
  '.'],
 ['fcc',
  'chairman',
  'ajit',
  'pai',
  'will',
  'step',
  'down',
  'on',
  'january',
  '20'],
 ['lpt',
  ':',
  'no',
  'matter',
  'how',
  'much',
  'your',
  'workplace',
  'pushes',
  '``',
  'team',
  'building',
  "''",
  'and',
  '``',
  'family',
  'culture',
  "''",
  '-',
  'remember',
  ',',
  'they',
  "'re",
  'not',
  'your',
  'friends',
  'and',
  'it',
  "'s",
  'still',
  'a',
  'workplace',
  '.'],
 ['mark',
  'kelly',
  'to',
  'be',
  'sworn',
  'in',
  'as',
  'senator',
  'on',
  'wednesday'],
 ['today',
  'i',
  'turn',
  '50',
  ',',
  'but',
  'here',
  'i',
  'am',
  'at',
  '18-',
  'summer',
  'of',
  '1989'],
 ['the',
  'kids',
  'have',
  'been',
  'asking'

In [50]:
pt = string.punctuation
pt += '...' + '``' + "''"
pt

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~...``\'\''

In [51]:
for i,doc in enumerate(docs):
    docs[i] = [word for word in doc if not word in pt]

In [52]:
docs

[['i',
  'could',
  "n't",
  'be',
  'more',
  'proud',
  'of',
  'her',
  'after',
  'a',
  'lot',
  'of',
  'blood',
  'sweat',
  'and',
  'tears',
  'but',
  'mostly',
  'fun',
  'my',
  'mother',
  'finished',
  'her',
  'first',
  'dark',
  'souls',
  'game'],
 ['fcc',
  'chairman',
  'ajit',
  'pai',
  'will',
  'step',
  'down',
  'on',
  'january',
  '20'],
 ['lpt',
  'no',
  'matter',
  'how',
  'much',
  'your',
  'workplace',
  'pushes',
  'team',
  'building',
  'and',
  'family',
  'culture',
  'remember',
  'they',
  "'re",
  'not',
  'your',
  'friends',
  'and',
  'it',
  "'s",
  'still',
  'a',
  'workplace'],
 ['mark',
  'kelly',
  'to',
  'be',
  'sworn',
  'in',
  'as',
  'senator',
  'on',
  'wednesday'],
 ['today',
  'i',
  'turn',
  '50',
  'but',
  'here',
  'i',
  'am',
  'at',
  '18-',
  'summer',
  'of',
  '1989'],
 ['the',
  'kids',
  'have',
  'been',
  'asking',
  'dad',
  'is',
  'delivering',
  'christmas',
  'is',
  'going',
  'to',
  'be',
  'great'],


In [63]:
sw = stopwords.words('english')

In [66]:
more_sw = ["n't"]
sw += more_sw
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [67]:
for i,doc in enumerate(docs):
    docs[i] = [token for token in doc if token not in sw]

In [68]:
docs

[['could',
  'proud',
  'lot',
  'blood',
  'sweat',
  'tears',
  'mostly',
  'fun',
  'mother',
  'finished',
  'first',
  'dark',
  'souls',
  'game'],
 ['fcc', 'chairman', 'ajit', 'pai', 'step', 'january', '20'],
 ['lpt',
  'matter',
  'much',
  'workplace',
  'pushes',
  'team',
  'building',
  'family',
  'culture',
  'remember',
  "'re",
  'friends',
  "'s",
  'still',
  'workplace'],
 ['mark', 'kelly', 'sworn', 'senator', 'wednesday'],
 ['today', 'turn', '50', '18-', 'summer', '1989'],
 ['kids', 'asking', 'dad', 'delivering', 'christmas', 'going', 'great'],
 ['100', 'control', 'bike', 'obstacle', 'stop'],
 ["'s"],
 ['hes', 'solved', 'case', 'move'],
 ['til',
  'real',
  'life',
  'top',
  'gun',
  'flight',
  'school',
  'became',
  'sick',
  'tired',
  'constant',
  'movie',
  'references',
  'imposed',
  '5',
  'fine',
  'staff',
  'member',
  'quotes',
  'movie'],
 ['oc', 'usa', 'covid', 'cases', 'syringe'],
 ['grow', 'fast'],
 ['madlad', '....'],
 ['hope', 'enjoys', 'braggin

In [72]:
lemmatizer = WordNetLemmatizer()

In [84]:
for i,doc in enumerate(docs):
    docs[i] = [lemmatizer.lemmatize(token) for token in doc]

In [85]:
docs

[['could',
  'proud',
  'lot',
  'blood',
  'sweat',
  'tear',
  'mostly',
  'fun',
  'mother',
  'finished',
  'first',
  'dark',
  'soul',
  'game'],
 ['fcc', 'chairman', 'ajit', 'pai', 'step', 'january', '20'],
 ['lpt',
  'matter',
  'much',
  'workplace',
  'push',
  'team',
  'building',
  'family',
  'culture',
  'remember',
  "'re",
  'friend',
  "'s",
  'still',
  'workplace'],
 ['mark', 'kelly', 'sworn', 'senator', 'wednesday'],
 ['today', 'turn', '50', '18-', 'summer', '1989'],
 ['kid', 'asking', 'dad', 'delivering', 'christmas', 'going', 'great'],
 ['100', 'control', 'bike', 'obstacle', 'stop'],
 ["'s"],
 ['he', 'solved', 'case', 'move'],
 ['til',
  'real',
  'life',
  'top',
  'gun',
  'flight',
  'school',
  'became',
  'sick',
  'tired',
  'constant',
  'movie',
  'reference',
  'imposed',
  '5',
  'fine',
  'staff',
  'member',
  'quote',
  'movie'],
 ['oc', 'usa', 'covid', 'case', 'syringe'],
 ['grow', 'fast'],
 ['madlad', '....'],
 ['hope', 'enjoys', 'bragging', 'right

In [86]:
# creating the vocabulary and initializing the BoW matrix
vocabulary = set()
for doc in docs:
    for token in doc:
        vocabulary.add(token)

vocabulary_lookup = {word: i for i, word in enumerate(vocabulary)}
matrix = np.zeros((len(docs), len(vocabulary)))

In [87]:
vocabulary_lookup

{'format': 0,
 'bugsnax': 1,
 'backed': 2,
 'recovered': 3,
 'andrzejrysuje': 4,
 'reynolds': 5,
 'sao': 6,
 'development': 7,
 'per': 8,
 'feyada': 9,
 'waffle': 10,
 'aoc': 11,
 '7th': 12,
 'nalgoticas': 13,
 'pokemon-related': 14,
 'chaotic': 15,
 'intoxicating': 16,
 'shredded': 17,
 'ne': 18,
 'redditors': 19,
 'escort': 20,
 'spotting': 21,
 'married': 22,
 '65': 23,
 'absolutelynotmeirl': 24,
 'mcdonalds': 25,
 'age': 26,
 'told': 27,
 '7d': 28,
 'monky': 29,
 'wok': 30,
 'dr': 31,
 'cheese': 32,
 'kq': 33,
 'wipe': 34,
 'luck': 35,
 'significant': 36,
 'next': 37,
 'anyone': 38,
 'barbara': 39,
 'sooner': 40,
 'guardian': 41,
 'stupidity': 42,
 'zonsopgang': 43,
 'massacre': 44,
 'pelt': 45,
 'wrestlemania': 46,
 '2018-2020': 47,
 'breakdance': 48,
 'easily': 49,
 'feeding': 50,
 'perfectly': 51,
 'devs': 52,
 'daddytm': 53,
 'intrusive': 54,
 '........': 55,
 'make': 56,
 'writer': 57,
 'others': 58,
 '101': 59,
 'actor': 60,
 '300+': 61,
 'peacekeeping': 62,
 'exalted': 63,
 

In [88]:
for doc_id, doc in enumerate(docs):
    for token in doc:
        word_id = vocabulary_lookup[token]
        matrix[doc_id][word_id] += 1

In [89]:
columns = sorted(vocabulary_lookup, key=lambda key: vocabulary_lookup[key])
df = pd.DataFrame(matrix.astype('int'), columns=columns)

In [90]:
df

,format,bugsnax,backed,recovered,andrzejrysuje,reynolds,sao,development,per,feyada,...,contains,prevent,smug,mythra,killer,definetly,loading,steal,bullet,spray
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
corpus = posts['title'].tolist()

In [77]:
vectorizer = TfidfVectorizer()

In [78]:
X = vectorizer.fit_transform(corpus)

In [79]:
X

<3000x7014 sparse matrix of type '<class 'numpy.float64'>'
	with 24142 stored elements in Compressed Sparse Row format>

In [80]:
vectorizer.get_feature_names()

['00',
 '000',
 '049',
 '08',
 '096',
 '0_o',
 '10',
 '100',
 '1000',
 '1000000',
 '100people',
 '100th',
 '101',
 '109',
 '11',
 '1114',
 '119',
 '11am',
 '12',
 '120',
 '125',
 '1280x1600',
 '13',
 '1300',
 '136',
 '13th',
 '14',
 '1400',
 '144',
 '1440x2560',
 '14s_data',
 '15',
 '150',
 '155',
 '1599x1006',
 '16',
 '160',
 '169',
 '16f',
 '16m',
 '17',
 '18',
 '180',
 '1800s',
 '1888',
 '18m',
 '19',
 '1900s',
 '1905',
 '1906',
 '1909',
 '1911',
 '1912',
 '1928',
 '1940',
 '1942',
 '1944',
 '1946',
 '1948',
 '1950s',
 '1958',
 '1964',
 '1967',
 '1970',
 '1973',
 '1982',
 '1983',
 '1984',
 '1985',
 '1987',
 '1989',
 '1990',
 '1992',
 '1997',
 '1998',
 '1999',
 '1kg',
 '1s',
 '1st',
 '1war',
 '20',
 '200',
 '2000s',
 '2001',
 '2002',
 '2009',
 '2010',
 '201130',
 '2012',
 '2013',
 '2014',
 '2016',
 '2018',
 '2019',
 '2020',
 '2021',
 '20k',
 '20th',
 '21',
 '212',
 '2143x1435',
 '21m',
 '22',
 '220',
 '23',
 '234',
 '2392',
 '24',
 '25',
 '250',
 '26',
 '28',
 '29',
 '291',
 '292',
 

In [82]:
print(X)

  (0, 2569)	0.17673585871153882
  (0, 5737)	0.2695299871681028
  (0, 1636)	0.224585560701923
  (0, 2387)	0.156171286886135
  (0, 2373)	0.21426111540188203
  (0, 4070)	0.2398233371409817
  (0, 4129)	0.10379652846442876
  (0, 2540)	0.19178890216879216
  (0, 4069)	0.2695299871681028
  (0, 1031)	0.14270004741459097
  (0, 6120)	0.2470577739350129
  (0, 422)	0.1061241194885063
  (0, 6039)	0.2695299871681028
  (0, 843)	0.25638458511853546
  (0, 3702)	0.21144015865235571
  (0, 321)	0.15819823128250182
  (0, 2910)	0.33034452129536174
  (0, 4341)	0.20529076415011466
  (0, 4848)	0.21426111540188203
  (0, 4062)	0.1693166889357023
  (0, 712)	0.13652003040257144
  (0, 1487)	0.22891472197174975
  (1, 80)	0.3016603170791322
  (1, 3272)	0.3476443460102773
  (1, 4368)	0.17801189987244728
  :	:
  (2996, 414)	0.2865719988512907
  (2996, 3897)	0.3499830294435984
  (2996, 3358)	0.25589930605309724
  (2996, 2107)	0.4575698168148259
  (2996, 4260)	0.2892411195731948
  (2997, 6015)	0.47641316468800216
  (2997,